In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib as mpl

import cmocean as cmx
import cmasher as cmr

from astropy.io import fits
from astropy.wcs import WCS
from astropy.visualization.wcsaxes import add_scalebar
import matplotlib.font_manager as fm
import astropy.units as u
import astropy.constants as c

from scipy.optimize import curve_fit, least_squares
import scipy.interpolate as sint
import scipy.special as sspec
import scipy.stats as sstats

from multiprocess import Pool
from functools import partial
import tqdm

import pandas as pd
import datashader as ds
from datashader.mpl_ext import dsshow

rnd = np.random.default_rng()
fontprops = fm.FontProperties(size=16)
nproc = 8

### Loading in the data
The data here come from the Herschel Gould Belt Survey (Andre+2010). In particular, this notebook is for the example for the Taurus L1495 region. The data come from http://www.herschel.fr/cea/gouldbelt/en/Phocea/Vie_des_labos/Ast/ast_visu.php?id_ast=66. You can download using
```
wget http://www.herschel.fr/Images/astImg/66/HGBS_taurus_L1495_column_density_map.fits.gz
```
and put it into a local data directly (here assumed to be `./data/`).

In [ ]:
cloudName = "taurus"
fitFile = "data/HGBS_taurus_L1495_column_density_map.fits"
df = fits.open(fitFile)
df.info()

The data is then trimmed a bit to get rid of extra white space and the WCS is stored for visualizations when wanted with matplotlib using the astropy matplotlib projection helpers.

In [ ]:
colData = df[0].data
if cloudName == "taurus":
    dataZoom = colData[850:6600, 880:7540]
else:
    dataZoom = colData.copy()

wcs = WCS(df[0].header)

In [ ]:
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111, projection=wcs)

im1 = ax.imshow(dataZoom, cmap=cmr.lilac, norm=colors.LogNorm(vmin=1e20, vmax=5e21))
ax.coords.grid(True, color='k', ls='solid')
ax.axis(False)
ax.set_aspect('equal')
cb1 = fig.colorbar(im1, shrink=0.75)
cb1.set_label("N(H) (cm$^{-2}$)", fontsize=16)
cb1.ax.tick_params(labelsize=14)

gc_distance = 140 * u.pc
scalebar_lenght = 2 * u.pc
scalebar_angle = (scalebar_lenght / gc_distance).to(
    u.deg, equivalencies=u.dimensionless_angles()
)

add_scalebar(ax, scalebar_angle, label="2 pc", color="k", fontproperties=fontprops)

plt.tight_layout()
plt.show()

Since the data size can be quite a bit, only keep the data that is needed for the calculation.

In [ ]:
colDataFlattened = colData[colData > 1E19].flatten()
del colData, df

Rather than do some optimization procedure to fit the lower column density end of the N-PDF for the log normal, here it was just done by eye. This is not the most optimal way, but will likely not be the dominant source of bias or uncertainty anyways in the final result given the simplicity of the model.

In [ ]:
N0 = 0.75*np.mean(colDataFlattened)
Nspace = np.logspace(19, 22.5, 128) #np.logspace(19, 23, 128)
xspace = Nspace/N0
zetaSpace = np.log(xspace)

x = colDataFlattened/N0
zeta = np.log(x)
stdGuess = np.std(zeta)

histZeta, bin_edges = np.histogram(zeta, bins='auto', density=True)
bin_centers = (bin_edges[:-1] + bin_edges[1:])/2

In [ ]:
def lognormal(x, sigma):
    return np.exp(-0.5*(x/sigma)**2)/(sigma*np.sqrt(2*np.pi))

stdFit = 0.375
print(N0, stdFit)

fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111)
ax.step(bin_centers, histZeta, where='mid', color='k', label='Data')
ax.plot(zetaSpace, lognormal(zetaSpace, stdFit), 'b-', label='Fit, $N_0 = %.2e$ cm$^{-2}$, $\sigma = %.2f$'%(N0, stdFit))
ax.set_xlabel(r"$\zeta = N/N_0",fontsize=16)
ax.set_ylabel("Number of Pixels", fontsize=16)
ax.legend(loc=2, fontsize=14)
ax.tick_params(which='both', axis='both', labelsize=14)
plt.show()

In [ ]:
del colDataFlattened, x, zeta

### Define the cloud macroscopic properties
We use typical cold cloud sizes and the results from the N-PDF fitting for the Mach number following the GMC clouds from Burkhart & Lazarian (2012). The cloud thicknesses should ideally come from newer Gaia results - we use 1 pc which is roughly consistent with the results from Qian+2015 and the newer 3D constraints from Zucker+2021. Using these, compute the transition density between the sonic scale and Jean's length.

In [ ]:
Ms = np.sqrt((np.exp(stdFit**2 / (0.16)) - 1.)/(0.5)**2)
T = 15.0 * u.K
N0u = N0 * u.cm**-2
if cloudName == 'taurus': 
    L = 1.0*u.pc
elif cloudName == 'orionB':
    L = 10.0*u.pc
elif cloudName == 'polaris':
    L = 2.5*u.pc
else:
    L = 2.5*u.pc

mu = 1.4
cs = np.sqrt(c.k_B*T/(mu*c.m_p))
sigma_Turb = stdFit
sigma_vel = Ms*cs

rhoJ = (np.pi*cs**2*Ms**4)/(c.G * L**2) #From Burkhart & Mocz 2019
nJ = rhoJ/(mu*c.m_p)
nJ = nJ.decompose().to(u.cm**(-3)) 

print(Ms, T, sigma_Turb, sigma_vel.decompose().to(u.km/u.s), nJ)

### Now starting the calculation...
First, defining the number of bins which gets the most likely density from the N-PDF histogram. These will be iterated on over time to better optimize - for now they're written to be relatively fast and work.

In [ ]:
dataDims = dataZoom.shape
nbinRes = 64

In [ ]:
#Clean the data in case there are any issues, just in case...
dataZoom[np.isnan(dataZoom)] = 0.0

In [ ]:
Nsamp = int(1E3)

Ndata = dataDims[0]*dataDims[1]
modelITurbCol = np.zeros(dataDims)
modelIITurbCol = np.zeros(dataDims)

def turb_dens_col_func(k):
    dataDims = dataZoom.shape
    i = k // dataDims[1]
    j = k % dataDims[1]
    Nobs = dataZoom[i,j]
    if np.isnan(dataZoom[i,j]):
        print("NAN DATA")
    if Nobs <= 0:
        return None, None
    zetaMax = np.log(Nobs/N0)
    minspace = min(np.log(0.01*Nobs/N0), -20*sigma_Turb)

    a, b = minspace / sigma_Turb, zetaMax / sigma_Turb
    try:
        r = sstats.truncnorm.rvs(a, b, loc=0.0, scale=sigma_Turb, size=Nsamp)
    except ValueError:
        print(Nobs, N0, sigma_Turb, a, b)
    NT_obs = (N0*np.exp(r))

    hist, bin_edges = np.histogram(np.log10(NT_obs), bins=nbinRes, density=True)
    bin_centers = (bin_edges[:-1] + bin_edges[1:])/2

    atc = 10**(bin_centers[np.argmax(hist)])

    return atc, np.amax(NT_obs)

with Pool(processes=nproc) as p:
    batno = 0
    numBatch = 10
    nBatch = Ndata // numBatch
    k = 0
    while batno < numBatch:
        lower = batno*nBatch
        upper = (batno+1)*nBatch
        if batno == numBatch-1:
            upper = Ndata
        results = list(
            tqdm.tqdm(
                p.imap(turb_dens_col_func,
                    np.arange(lower, upper, 1), dataDims[1]),
                total=(upper - lower)
            )
        ) 
        for atci, matci in results:
            if atci is None:
                k += 1
                continue
            i = k // dataDims[1]
            j = k % dataDims[1]
            modelITurbCol[i,j] = atci
            modelIITurbCol[i,j] = matci
            k += 1
        del results
        batno += 1
    p.close()
    p.join()

In [ ]:
n_range = np.logspace(-2, 9, 128) * u.cm**-3
x_range = n_range/nJ

y = x_range**(1.5)/(1 + x_range)
dN = y*(np.sqrt(np.pi/(c.G*mu*c.m_p))*cs*nJ**(1/2)).decompose().to(u.cm**-2)
dN_n_interp = sint.interp1d(dN.value, n_range.value, kind='linear', fill_value='extrapolate', bounds_error=False)

In [ ]:
modelIGravDens = np.zeros(dataDims)
modelITurbDens = np.zeros(dataDims)

def avgGravDens_func(k):
    i = k // dataDims[1]
    j = k % dataDims[1]
    if dataZoom[i,j] <= 0:
        return None, None
    atc = modelITurbCol[i,j]
    dNi = dataZoom[i,j] - atc
    if dNi <= 0:
        agd = 0.0
    else:
        agd = dN_n_interp(dNi)

    xi = agd/(nJ.value)
    switchi = xi/(1 + xi)
    lambdai = (np.sqrt((np.pi*cs**2)/(c.G*mu*c.m_p))*(agd*(u.cm**(-3)))**(-1/2)).decompose().to(u.pc)
    atd = atc/((L - switchi*lambdai).to(u.cm).value)

    return atd, agd

with Pool(processes=nproc) as p:
    batno = 0
    numBatch = 10
    nBatch = Ndata // numBatch
    k = 0
    while batno < numBatch:
        lower = batno*nBatch
        upper = (batno+1)*nBatch
        if batno == numBatch-1:
            upper = Ndata
        results = list(
            tqdm.tqdm(
                p.imap(avgGravDens_func ,
                    np.arange(lower, upper, 1), dataDims[1]),
                total=(upper - lower)
            )
        ) 
        for atd, agd in results:
            if atd is None:
                k += 1
                continue
            i = k // dataDims[1]
            j = k % dataDims[1]
            modelITurbDens[i,j] = atd
            modelIGravDens[i,j] = agd
            k += 1
        del results
        batno += 1
    p.close()
    p.join()

In [ ]:
modelIIGravDens = np.zeros((dataDims[0], dataDims[1]))
modelIITurbDens = np.zeros((dataDims[0], dataDims[1]))

def sampGravDens_func(k):
    i = k // dataDims[1]
    j = k % dataDims[1]
    if dataZoom[i,j] <= 0:
        return None, None
    atc = modelIITurbCol[i,j]
    dNi = dataZoom[i,j] - atc
    if dNi <= 0:
        agd = 0.0
    else:
        agd = dN_n_interp(dNi)

    xi = agd/(nJ.value)
    switchi = xi/(1 + xi)
    lambdai = (np.sqrt((np.pi*cs**2)/(c.G*mu*c.m_p))*(agd*(u.cm**(-3)))**(-1/2)).decompose().to(u.pc)
    atd = atc/((L - switchi*lambdai).to(u.cm).value)

    return atd, agd

with Pool(processes=nproc) as p:
    batno = 0
    numBatch = 10
    nBatch = Ndata // numBatch
    k = 0
    while batno < numBatch:
        print("On batch no: %d"%batno)
        lower = batno*nBatch
        upper = (batno+1)*nBatch
        if batno == numBatch-1:
            upper = Ndata
        results = list(
            tqdm.tqdm(
                p.imap(sampGravDens_func ,
                    np.arange(lower, upper, 1), dataDims[1]),
                total=(upper - lower)
            )
        ) 
        for atd, agd in results:
            if atd is None:
                k += 1
                continue
            i = k // dataDims[1]
            j = k % dataDims[1]
            modelIITurbDens[i,j] = atd
            modelIIGravDens[i,j] = agd
            k += 1
        del results
        batno += 1
    p.close()
    p.join()

In [ ]:
x_obs = modelIGravDens/(nJ.value)
switch_obs = x_obs/(1 + x_obs)
del x_obs
lambdaJ_map = (np.sqrt((np.pi*cs**2)/(c.G*mu*c.m_p))*(modelIGravDens*(u.cm**(-3)))**(-1/2)).decompose().to(u.pc)
lambdaJ_map[modelIGravDens <= 0] = 0.0*u.pc
modelIGravCol = modelIGravDens*switch_obs*lambdaJ_map.to(u.cm).value
lambdaJ_eff_modelI = switch_obs*lambdaJ_map.to(u.pc).value
del lambdaJ_map

In [ ]:
x_obs = modelIIGravDens/(nJ.value)
switch_obs = x_obs/(1 + x_obs)
del x_obs
lambdaJ_map = (np.sqrt((np.pi*cs**2)/(c.G*mu*c.m_p))*(modelIIGravDens*(u.cm**(-3)))**(-1/2)).decompose().to(u.pc)
lambdaJ_map[modelIIGravDens <= 0] = 0.0*u.pc
modelIIGravCol = modelIIGravDens*switch_obs*lambdaJ_map.to(u.cm).value
lambdaJ_eff_modelII = switch_obs*lambdaJ_map.to(u.pc).value
del lambdaJ_map

In [ ]:
Neff_map_modelI = np.zeros(dataDims)

def neff_func(k):
    dataDims = dataZoom.shape
    i = k // dataDims[1]
    j = k % dataDims[1]
    Nobs = dataZoom[i,j]
    if Nobs <= 0:
        return None
    zetaMax = np.log(Nobs/N0)
    minspace = min(np.log(0.01*Nobs/N0), -20*sigma_Turb)

    a, b = minspace / sigma_Turb, zetaMax / sigma_Turb
    try:
        r = sstats.truncnorm.rvs(a, b, loc=0.0, scale=sigma_Turb, size=Nsamp)
    except ValueError:
        print(Nobs, N0, sigma_Turb, a, b)
    randSpot = np.random.uniform(0, 1, Nsamp)
    NT_eff = 0.5*randSpot*(N0*np.exp(r))

    hist, bin_edges = np.histogram(np.log10(NT_eff), bins=nbinRes, density=True)
    bin_centers = (bin_edges[:-1] + bin_edges[1:])/2

    turbCol = 10**(bin_centers[np.argmax(hist)])

    neffi = turbCol + modelIGravCol[i,j]

    return neffi

with Pool(processes=nproc) as p:
    batno = 0
    numBatch = 10
    nBatch = Ndata // numBatch
    k = 0
    while batno < numBatch:
        print("On batch no: %d"%(batno+1))
        lower = batno*nBatch
        upper = (batno+1)*nBatch
        if batno == numBatch-1:
            upper = Ndata
        results = list(
            tqdm.tqdm(
                p.imap(neff_func,
                    np.arange(lower, upper, 1), dataDims[1]),
                total=(upper - lower)
            )
        )
        for neffi in results:
            if neffi is None:
                k += 1
                continue
            i = k // dataDims[1]
            j = k % dataDims[1]
            Neff_map_modelI[i,j] = neffi
            k += 1
        del results
        batno += 1
    p.close()
    p.join()

In [ ]:
Neff_map_modelII = np.zeros(dataDims)

def neff_funcSamp(k):
    dataDims = dataZoom.shape
    i = k // dataDims[1]
    j = k % dataDims[1]
    Nobs = dataZoom[i,j]
    if Nobs <= 0:
        return None
    zetaMax = np.log(Nobs/N0)
    minspace = min(np.log(0.01*Nobs/N0), -20*sigma_Turb)

    a, b = minspace / sigma_Turb, zetaMax / sigma_Turb
    try:
        r = sstats.truncnorm.rvs(a, b, loc=0.0, scale=sigma_Turb, size=Nsamp)
    except ValueError:
        print(Nobs, N0, sigma_Turb, a, b)
    randSpot = np.random.uniform(0, 1, Nsamp)
    NT_eff = 0.5*randSpot*(N0*np.exp(r))

    hist, bin_edges = np.histogram(np.log10(NT_eff), bins=nbinRes, density=True)
    bin_centers = (bin_edges[:-1] + bin_edges[1:])/2

    turbCol = 10**(bin_centers[np.argmax(hist)])

    neffi = turbCol + modelIIGravCol[i,j]

    return neffi

with Pool(processes=nproc) as p:
    batno = 0
    numBatch = 10
    nBatch = Ndata // numBatch
    k = 0
    while batno < numBatch:
        print("On batch no: %d"%(batno+1))
        lower = batno*nBatch
        upper = (batno+1)*nBatch
        if batno == numBatch-1:
            upper = Ndata
        results = list(
            tqdm.tqdm(
                p.imap(neff_funcSamp,
                    np.arange(lower, upper, 1), dataDims[1]),
                total=(upper - lower)
            )
        )
        for neffi in results:
            if neffi is None:
                k += 1
                continue
            i = k // dataDims[1]
            j = k % dataDims[1]
            Neff_map_modelII[i,j] = neffi
            k += 1
        del results
        batno += 1
    p.close()
    p.join()

In [ ]:
fontprops = fm.FontProperties(size=16)
modelITurbDens[dataZoom <= 0] = 0.0

modelITurbDens_plot = modelITurbDens.copy()
modelITurbDens_plot[modelITurbDens_plot <= 0] = np.nan

if cloudName == "taurus":
    vmin, vmax = 5e1, 500
elif cloudName == 'orionB':
    vmin, vmax = 5e0, 35.
else:
    vmin, vmax = 1e1, 75.

fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111, projection=wcs)

im1 = ax.imshow(modelITurbDens_plot, cmap=cmr.lilac, norm=colors.Normalize(vmin=vmin, vmax=vmax))
ax.axis(False)
ax.set_aspect('equal')
cb1 = fig.colorbar(im1, shrink=0.75)
cb1.set_label("n$_T$ (cm$^{-3}$)", fontsize=16)
cb1.ax.tick_params(labelsize=14)

gc_distance = 140 * u.pc
scalebar_lenght = 2 * u.pc
scalebar_angle = (scalebar_lenght / gc_distance).to(
    u.deg, equivalencies=u.dimensionless_angles()
)

# Add a scale bar
add_scalebar(ax, scalebar_angle, label="2 pc", color="k", fontproperties=fontprops)

plt.tight_layout()
fig.savefig("figures/turb_dens_modelI_%s.pdf"%cloudName, bbox_inches='tight')
plt.show()

del modelITurbDens_plot

In [ ]:
modelIGravDens_plot = modelIGravDens.copy()
modelIGravDens_plot[dataZoom <= 0] = np.nan

if cloudName == 'taurus':
    vmin, vmax = 1e3, 2e5
elif cloudName == 'orionB':
    vmin, vmax = 1e2, 4e4
else:
    vmin, vmax = 1e1, 5e3

fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111, projection=wcs)

im1 = ax.imshow(modelIGravDens_plot, cmap=cmr.lilac, norm=colors.LogNorm(vmin=vmin, vmax=vmax))
ax.axis(False)
ax.set_aspect('equal')
cb1 = fig.colorbar(im1, shrink=0.75)
cb1.set_label("n$_J$ (cm$^{-3}$)", fontsize=16)
cb1.ax.tick_params(labelsize=14)

gc_distance = 140 * u.pc
scalebar_lenght = 2 * u.pc
scalebar_angle = (scalebar_lenght / gc_distance).to(
    u.deg, equivalencies=u.dimensionless_angles()
)

# Add a scale bar
add_scalebar(ax, scalebar_angle, label="2 pc", color="k", fontproperties=fontprops)

plt.tight_layout()
fig.savefig("figures/grav_dens_modelI_%s.pdf"%cloudName, bbox_inches='tight')
plt.show()
del modelIGravDens_plot

In [ ]:
Neff_map_modelI[dataZoom <= 0] = np.nan
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111, projection=wcs)

if cloudName == 'taurus':
    vmin, vmax = 1e20, 1e22
elif cloudName == 'orionB':
    vmin, vmax = 5e19, 2.5e22
else:
    vmin, vmax = 5e19, 5e21

#im1 = ax.imshow(colData[850:6600, 880:7540], cmap=cmr.lilac, norm=colors.LogNorm(vmin=2.5e20, vmax=5e22))
im1 = ax.imshow(Neff_map_modelI, cmap=cmr.lilac, norm=colors.LogNorm(vmin=vmin, vmax=vmax))
#ax.coords.grid(True, color='k', ls='solid')
ax.axis(False)
ax.set_aspect('equal')
cb1 = fig.colorbar(im1, shrink=0.75)
cb1.set_label(r"N$_{\rm eff}$(H) (cm$^{-2}$)", fontsize=16)
cb1.ax.tick_params(labelsize=14)

gc_distance = 140 * u.pc
scalebar_lenght = 2 * u.pc
scalebar_angle = (scalebar_lenght / gc_distance).to(
    u.deg, equivalencies=u.dimensionless_angles()
)

# Add a scale bar
add_scalebar(ax, scalebar_angle, label="2 pc", color="k", fontproperties=fontprops)

plt.tight_layout()
fig.savefig("figures/neff_modelI_%s.pdf"%cloudName, bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111, projection=wcs)

#im1 = ax.imshow(colData[850:6600, 880:7540], cmap=cmr.lilac, norm=colors.LogNorm(vmin=2.5e20, vmax=5e22))
im1 = ax.imshow(Neff_map_modelI/dataZoom, cmap='magma', norm=colors.Normalize(vmin=0.1, vmax=1))
ax.axis(False)
ax.set_aspect('equal')
cb1 = fig.colorbar(im1, shrink=0.75)
cb1.set_label("Neff/N(H) (cm$^{-2}$)", fontsize=16)
cb1.ax.tick_params(labelsize=14)


plt.tight_layout()
fig.savefig("figures/neff_ratio_%s_modelI.pdf"%cloudName, bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111, projection=wcs)

im1 = ax.imshow(modelIGravCol/dataZoom, cmap='magma', norm=colors.Normalize(vmin=0.0, vmax=1))
ax.axis(False)
ax.set_aspect('equal')
cb1 = fig.colorbar(im1, shrink=0.75)
cb1.set_label("N$_J$/N(H) (cm$^{-2}$)", fontsize=16)
cb1.ax.tick_params(labelsize=14)


plt.tight_layout()
fig.savefig("figures/dens_gas_ratio_modelI_%s.pdf"%cloudName, bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111, projection=wcs)

#im1 = ax.imshow(colData[850:6600, 880:7540], cmap=cmr.lilac, norm=colors.LogNorm(vmin=2.5e20, vmax=5e22))
im1 = ax.imshow(modelITurbCol/dataZoom, cmap='magma', norm=colors.Normalize(vmin=0.0, vmax=1))
ax.axis(False)
ax.set_aspect('equal')
cb1 = fig.colorbar(im1, shrink=0.75)
cb1.set_label("N$_T$/N(H) (cm$^{-2}$)", fontsize=16)
cb1.ax.tick_params(labelsize=14)


plt.tight_layout()
fig.savefig("figures/turb_gas_ratio_modelI_%s.pdf"%cloudName, bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111, projection=wcs)

lambdaJ_eff_plot = lambdaJ_eff_modelI.copy()
lambdaJ_eff_plot[dataZoom <= 0] = np.nan

if cloudName == 'taurus':
    vmin, vmax = 1E-3, 5E-2
elif cloudName == 'orionB':
    vmin, vmax = 5E-3, 5e-2
else:
    vmin, vmax = 0.03, 0.17

#im1 = ax.imshow(colData[850:6600, 880:7540], cmap=cmr.lilac, norm=colors.LogNorm(vmin=2.5e20, vmax=5e22))
im1 = ax.imshow(lambdaJ_eff_plot, cmap='magma', norm=colors.LogNorm(vmin=vmin, vmax=vmax))
#ax.contour(avgGravCol/dataZoom, levels=[0.5], colors='w', linewidths=0.5)
ax.axis(False)
ax.set_aspect('equal')
cb1 = fig.colorbar(im1, shrink=0.75)
cb1.set_label(r"$\lambda_{\rm eff, J}$ (pc)", fontsize=16)
cb1.ax.tick_params(labelsize=14)


plt.tight_layout()
fig.savefig("figures/jean_length_modelI_%s.pdf"%cloudName, bbox_inches='tight')
plt.show()
del lambdaJ_eff_plot

In [ ]:
fontprops = fm.FontProperties(size=16)
modelIITurbDens[dataZoom <= 0] = 0.0

modelIITurbDens_plot = modelIITurbDens.copy()
modelIITurbDens_plot[modelIITurbDens_plot <= 0] = np.nan

if cloudName == "taurus":
    vmin, vmax = 5e1, 2.5e3
elif cloudName == 'orionB':
    vmin, vmax = 5e0, 35.
else:
    vmin, vmax = 1e1, 2.5e2

fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111, projection=wcs)

im1 = ax.imshow(modelIITurbDens_plot, cmap=cmr.lilac, norm=colors.LogNorm(vmin=vmin, vmax=vmax))
ax.axis(False)
ax.set_aspect('equal')
cb1 = fig.colorbar(im1, shrink=0.75)
cb1.set_label("n$_T$ (cm$^{-3}$)", fontsize=16)
cb1.ax.tick_params(labelsize=14)

gc_distance = 140 * u.pc
scalebar_lenght = 2 * u.pc
scalebar_angle = (scalebar_lenght / gc_distance).to(
    u.deg, equivalencies=u.dimensionless_angles()
)

# Add a scale bar
add_scalebar(ax, scalebar_angle, label="2 pc", color="k", fontproperties=fontprops)

plt.tight_layout()
fig.savefig("figures/turb_dens_modelII_%s.pdf"%cloudName, bbox_inches='tight')
plt.show()

del modelIITurbDens_plot

In [ ]:
modelIIGravDens_plot = modelIIGravDens.copy()
modelIIGravDens_plot[dataZoom <= 0] = np.nan

if cloudName == 'taurus':
    vmin, vmax = 1e1, 2e5
elif cloudName == 'orionB':
    vmin, vmax = 1e2, 4e4
else:
    vmin, vmax = 1e0, 5e3

fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111, projection=wcs)

im1 = ax.imshow(modelIIGravDens_plot, cmap=cmr.lilac, norm=colors.LogNorm(vmin=vmin, vmax=vmax))
ax.axis(False)
ax.set_aspect('equal')
cb1 = fig.colorbar(im1, shrink=0.75)
cb1.set_label("n$_J$ (cm$^{-3}$)", fontsize=16)
cb1.ax.tick_params(labelsize=14)

gc_distance = 140 * u.pc
scalebar_lenght = 2 * u.pc
scalebar_angle = (scalebar_lenght / gc_distance).to(
    u.deg, equivalencies=u.dimensionless_angles()
)

# Add a scale bar
add_scalebar(ax, scalebar_angle, label="2 pc", color="k", fontproperties=fontprops)

plt.tight_layout()
fig.savefig("figures/grav_dens_modelII_%s.pdf"%cloudName, bbox_inches='tight')
plt.show()
del modelIIGravDens_plot

In [ ]:
#Save the 2D arrays as numpy arrays
np.save("data/ModelITurbDens_%s.npy"%cloudName, modelITurbDens)
np.save("data/ModelIGravDens_%s.npy"%cloudName, modelIGravDens)
np.save("data/ModelIITurbDens_%s.npy"%cloudName, modelIITurbDens)
np.save("data/ModelIIGravDens_%s.npy"%cloudName, modelIIGravDens)
np.save("data/ModelINeff_%s.npy"%cloudName, Neff_map_modelI)
np.save("data/ModelIINeff_%s.npy"%cloudName, Neff_map_modelII)
np.save("data/ModelIlambdaJ_eff_%s.npy"%cloudName, lambdaJ_eff_modelI)
np.save("data/ModelIIlambdaJ_eff_%s.npy"%cloudName, lambdaJ_eff_modelII)
np.save("data/ModelITurbCol_%s"%cloudName, modelITurbCol)
np.save("data/ModelIGravCol_%s"%cloudName, modelIGravCol)
np.save("data/ModelIITurbCol_%s"%cloudName, modelIITurbCol)
np.save("data/ModelIIGravCol_%s"%cloudName, modelIIGravCol)